#### Import library and setup notebook

In [1]:
import pandas as pd
import re
import unicodedata

# Show wider options
pd.set_option('display.max_colwidth', 150)

#### Clean data, standardize Vietnamese (remove accents, stop words)

##### Text cleaning function

In [2]:
def clean_text(text):
    text = str(text).lower()
    text = unicodedata.normalize('NFC', text) 
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    text = re.sub(r"[^0-9a-zA-ZÀ-Ỵà-ỵĐđ\s]", ' ', text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

1. **Chuyển thành chữ thường:** giúp thống nhất dữ liệu (`"Tốt"` → `"tốt"`).  
2. **Xóa liên kết (URL):** loại bỏ các đường dẫn web (`https://...`).  
3. **Xóa ký tự đặc biệt:** chỉ giữ lại chữ, số và ký tự tiếng Việt.  
4. **Xóa khoảng trắng thừa:** gom nhiều khoảng trắng thành một, xóa đầu/cuối.

##### Remove stopwords function

In [3]:
def remove_stopwords(text, stopwords):
    words = text.split()  # Split text into list words.
    
    filtered = [w for w in words if w not in stopwords]  # Filter words that are not stopwords.
    
    return " ".join(filtered)  # Concatenate filtered to string

1. **Tách câu thành danh sách từ:** (`"tôi thích uống trà"` → `["tôi", "thích", "uống", "trà"]`)
2. **Lọc bỏ từ dừng:** giữ lại những từ không nằm trong danh sách stopwords.
3. **Ghép lại thành chuỗi:** (`["thích", "uống", "trà"]` → `"thích uống trà"`)

##### Download the list of stop words

In [5]:
stopwords = set(open("D:/Projects/Sentiment_Analysis/data/external/vietnamese-stopwords.txt", "r", encoding="utf-8").read().splitlines())
emotion_words = [
    # Negation, degree
    "không", "chẳng", "chả", "chưa", "chẳng", "đầy", "đủ", "rất", "hơi", "khá", "cực", "quá",
    "vô cùng", "hết sức", "cực kỳ", "siêu", "vô đối", "hơi", "khá", "tương đối",

    # Positive
    "tốt", "đẹp", "hay", "tuyệt", "xuất sắc", "đỉnh", "ổn", "vui", "ưng", "dễ thương", "đáng yêu",
    "mượt", "chất lượng", "ấn tượng", "hiệu quả", "thích", "hấp dẫn", "hoàn hảo", "đáng tin",
    "đáng giá", "chuẩn", "xịn", "tuyệt vời", "ổn áp", "ngon", "ngon lành", "ok", "oke",

    # Negative
    "tệ", "xấu", "kém", "chán", "ghét", "bực", "dở", "tồi", "khó chịu", "kinh khủng",
    "thất vọng", "chậm", "lỗi", "rác", "ngu", "bực mình", "quá đáng", "bực tức", "dở ẹc",
    "chán đời", "kệch cỡm", "vô dụng", "phiền", "tồi tệ", "dở dang"
]
for w in emotion_words:
    stopwords.discard(w)

#### Read and Process data

In [6]:
# Read each episode
train_df = pd.read_csv("D:/Projects/Sentiment_Analysis/data/interim/train.csv", encoding="utf-8-sig")
test_df  = pd.read_csv("D:/Projects/Sentiment_Analysis/data/interim/test.csv", encoding="utf-8-sig")
dev_df   = pd.read_csv("D:/Projects/Sentiment_Analysis/data/interim/dev.csv", encoding="utf-8-sig")

# Apply the same handler function to all three
for df in [train_df, test_df, dev_df]:
    df["sentence"] = df["sentence"].apply(clean_text)
    df["sentence"] = df["sentence"].apply(lambda x: remove_stopwords(x, stopwords))

# Save
train_df.to_csv("D:/Projects/Sentiment_Analysis/data/processed/train_clean.csv", index=False, encoding="utf-8-sig")
test_df.to_csv("D:/Projects/Sentiment_Analysis/data/processed/test_clean.csv", index=False, encoding="utf-8-sig")
dev_df.to_csv("D:/Projects/Sentiment_Analysis/data/processed/dev_clean.csv", index=False, encoding="utf-8-sig")